In [ ]:
# 모듈 임포트
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import matplotlib.pyplot as plt

In [ ]:
# 1. 정상 데이터 불러오기
column_names = ['timestamp', 'v_raw', 'c_raw', 'voltage', 'current']
normal_df = pd.read_csv('../realtime/raw_dataset_normal_90sec.csv', names=column_names, header=None)
normal_df['label'] = 0  # 정상 라벨 추가

# 2. 아크(이상) 데이터 불러오기
# → 아크 데이터도 같은 형식의 CSV라고 가정
arc_df = pd.read_csv('../realtime/raw_dataset_arc_30sec.csv', names=column_names, header=None)
arc_df['label'] = 1  # 이상 라벨 추가

# 3. 두 데이터 합치기
train_df = pd.concat([normal_df, arc_df], ignore_index=True)


In [ ]:
# 파생 피처 생성
def add_features(df):
    df['voltage_diff'] = df['voltage'].diff().fillna(0).abs()
    df['current_diff'] = df['current'].diff().fillna(0).abs()
    df['voltage_ma'] = df['voltage'].rolling(5).mean().bfill()
    df['current_ma'] = df['current'].rolling(5).mean().bfill()
    df['power'] = df['voltage'] * df['current']
    df['power_diff'] = df['power'].diff().fillna(0).abs()
    return df

train_df = add_features(train_df).dropna()

In [ ]:
# 입력/정답 분리
features = ['voltage', 'current', 'voltage_diff', 'current_diff',
            'voltage_ma', 'current_ma', 'power', 'power_diff']
X = train_df[features].values
y = train_df['label'].values

# 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_scaled, y)

# 저장
joblib.dump(model, './model/random_forest_model.joblib')
joblib.dump(scaler, './model/rf_scaler.joblib')

print("✅ 모델 학습 및 저장 완료!")